# IEOR 4703 -- Monte Carlo Simulation for FE

## Delayed Start options via simulation w/o and w/ conditional MC

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from time import time
import math

In [ ]:
# import our BMS price and delta functions
from BMS import BMS_price, BMS_delta
# import ecdf function
from statsmodels.distributions.empirical_distribution import ECDF

In [ ]:
s0 = 100
t1 = 0.5
T = 2
sig = 0.30
r = 0.04
q = 0.015

lda = 0.9

In [ ]:
# Exact solution (derivation shown in the notes)
c = s0 * np.exp(-q*t1) * BMS_price('call', 1, lda, r, q, sig, T, t1)
print('Exact solution: %3.4f' % c)

Exact solution: 20.6358


In [ ]:
np.random.seed(623456)
n_sims = 100000
discountedPayoffs1 = np.zeros(n_sims)
discountedPayoffs2 = np.zeros(n_sims)

In [ ]:
st = time()

# could be made faster by vectorizing and removing the for loop
for i in range(n_sims):
    z1 = np.random.randn()
    s1 = s0 * np.exp((r - q -sig*sig/2)*t1 + sig*np.sqrt(t1)*z1)
    # set strike
    K = lda * s1
    # naive MC
    z2 = np.random.randn()
    s2 = s1 * np.exp((r - q -sig*sig/2)*(T - t1) + sig*np.sqrt(T - t1)*z2)
    discountedPayoffs1[i] = np.exp(-r*T) * max(s2 - K, 0)
    # conditional MC
    discountedPayoffs2[i] = np.exp(-r*t1) * BMS_price('call', s1, K, r, q, sig, T, t1)

c_hat1 = np.mean(discountedPayoffs1)
var_hat1 = np.var(discountedPayoffs1, ddof=1)

c_hat2 = np.mean(discountedPayoffs2)
var_hat2 = np.var(discountedPayoffs2, ddof=1)

et = time()
print('Elapsed time was %3.4f seconds' % (et - st))

Elapsed time was 19.6904 seconds


### Pay attention to the half-width of CI

In [ ]:
print('Exact solution: %3.4f' % c)
print('n_sims = %i' % n_sims)
print('Naive MC: %3.4f (%3.4f)' % (c_hat1, math.sqrt(var_hat1/n_sims)))
print('Conditional MC: %3.4f (%3.4f)' % (c_hat2, math.sqrt(var_hat2/n_sims)))

Exact solution: 20.6358
n_sims = 100000
Naive MC: 20.4432 (0.0968)
Conditional MC: 20.6283 (0.0140)
